In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Input, Embedding, GRU, Dense, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import gensim.downloader as api

In [2]:
# Load dataset
file_path = "/Users/celinewu/Documents/GitHub/2024-25c-fai2-adsai-group-group16/Task_4/ver_2_FINAL_DATASET.xlsx"
df = pd.read_excel(file_path)

In [3]:
# Extract text and labels
sentences = df["Sentence"].astype(str).tolist()
labels = df["main_category"].astype(str).tolist()

# Extract Sentiment Scores
sentiment_scores = df["Sentiment_Score"].values.reshape(-1, 1)

# Extract TF-IDF Feature as an Additional Feature
tfidf_features = np.array([np.fromstring(vec.strip("[]"), sep=' ') for vec in df["TF_IDF"]])

# Normalize Features
scaler = MinMaxScaler()
sentiment_scores = scaler.fit_transform(sentiment_scores)
tfidf_features = scaler.fit_transform(tfidf_features)

/var/folders/kz/slhyr0gd4qj4nrryyswvqv4c0000gn/T/ipykernel_68131/2282603795.py:9: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  tfidf_features = np.array([np.fromstring(vec.strip("[]"), sep=' ') for vec in df["TF_IDF"]])


In [4]:
# Tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(sentences)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding="post")


In [5]:
# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)


In [6]:
# Split dataset
X_train, X_test, y_train, y_test, X_train_sent, X_test_sent, X_train_tfidf, X_test_tfidf = train_test_split(
    padded_sequences, encoded_labels, sentiment_scores, tfidf_features,
    test_size=0.2, random_state=42, stratify=encoded_labels
)

# Convert labels to categorical
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)


In [7]:
# Load GloVe embeddings
print("Loading GloVe embeddings...")
glove_model = api.load("glove-wiki-gigaword-100")

# Create embedding matrix
embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    if word in glove_model:
        embedding_matrix[i] = glove_model[word]

# Define text input and embedding layer
text_input = Input(shape=(max_length,), name="text_input")
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_dim, 
                      weights=[embedding_matrix], input_length=max_length, 
                      trainable=False)(text_input)


Loading GloVe embeddings...


/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
2025-03-04 09:40:51.515437: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2025-03-04 09:40:51.515477: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-03-04 09:40:51.515481: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-03-04 09:40:51.515696: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-03-04 09:40:51.515712: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefin

In [8]:
# Define GRU layers
rnn_layer = GRU(128, return_sequences=True, recurrent_dropout=0.3)(embedding)
rnn_layer = GRU(64, recurrent_dropout=0.3)(rnn_layer)
dense_text = Dense(64, activation='relu')(rnn_layer)

# Define Sentiment Score input
sentiment_input = Input(shape=(1,), name="sentiment_input")
sentiment_dense = Dense(8, activation='relu')(sentiment_input)

# Define TF-IDF input
tfidf_input = Input(shape=(tfidf_features.shape[1],), name="tfidf_input")
tfidf_dense = Dense(32, activation='relu')(tfidf_input)


In [9]:
# Merge inputs
merged = Concatenate()([dense_text, sentiment_dense, tfidf_dense])
output = Dense(num_classes, activation='sigmoid')(merged)


In [10]:
# Define final model
model = Model(inputs=[text_input, sentiment_input, tfidf_input], outputs=output)
optimizer = Adam(learning_rate=0.0005)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(encoded_labels), y=encoded_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}


In [11]:
# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint("sentiment_tfidf_it3.keras", monitor='val_accuracy', save_best_only=True)

# Train the model
print("Training model...")
history = model.fit(
    [X_train, X_train_sent, X_train_tfidf], y_train,
    epochs=25, batch_size=32, validation_data=([X_test, X_test_sent, X_test_tfidf], y_test),
    class_weight=class_weight_dict,
    callbacks=[early_stopping, model_checkpoint]
)

Training model...
Epoch 1/25


2025-03-04 09:40:52.679238: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


140/140 ━━━━━━━━━━━━━━━━━━━━ 390s 3s/step - accuracy: 0.1367 - loss: 7.8302 - val_accuracy: 0.1434 - val_loss: 0.5816
Epoch 2/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 387s 3s/step - accuracy: 0.1538 - loss: 0.5485 - val_accuracy: 0.1434 - val_loss: 0.4653
Epoch 3/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 384s 3s/step - accuracy: 0.1356 - loss: 0.4487 - val_accuracy: 0.1720 - val_loss: 0.4215
Epoch 4/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 423s 3s/step - accuracy: 0.1405 - loss: 0.4208 - val_accuracy: 0.1568 - val_loss: 0.4132
Epoch 5/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 420s 3s/step - accuracy: 0.1354 - loss: 0.4138 - val_accuracy: 0.1595 - val_loss: 0.4116
Epoch 6/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 380s 3s/step - accuracy: 0.1354 - loss: 0.4136 - val_accuracy: 0.1595 - val_loss: 0.4110
Epoch 7/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 378s 3s/step - accuracy: 0.1660 - loss: 0.4112 - val_accuracy: 0.1559 - val_loss: 0.4105
Epoch 8/25
140/140 ━━━━━━━━━━━━━━━━━━━━ 380s 3s/step - accuracy: 0.1565 - loss: 0.4109 - val_accuracy: 0.161

In [12]:
# Evaluate the model
test_loss, test_acc = model.evaluate([X_test, X_test_sent, X_test_tfidf], y_test)
print(f"Test Accuracy: {test_acc:.4f}")


35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 133ms/step - accuracy: 0.2261 - loss: 0.4030
Test Accuracy: 0.2159


In [13]:
# Save tokenizer
import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


In [14]:
# Generate F1-score report
from sklearn.metrics import classification_report

# Get predictions
y_pred = model.predict([X_test, X_test_sent, X_test_tfidf])

# Convert predictions from one-hot encoding to class indices
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Generate classification report
print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))


35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 139ms/step

Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       159
     disgust       0.30      0.47      0.36       159
        fear       0.00      0.00      0.00       159
   happiness       0.19      0.89      0.31       160
     neutral       0.00      0.00      0.00       160
     sadness       0.20      0.15      0.17       159
    surprise       0.00      0.00      0.00       160

    accuracy                           0.22      1116
   macro avg       0.10      0.22      0.12      1116
weighted avg       0.10      0.22      0.12      1116



/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()

In [ ]:
# Generate F1-score report
from sklearn.metrics import classification_report

# Get predictions
y_pred = model.predict([X_test, X_test_sent, X_test_tfidf])

# Convert predictions from one-hot encoding to class indices
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_test, axis=1)

# Generate classification report
print("\nClassification Report:")
print(classification_report(y_test_classes, y_pred_classes, target_names=label_encoder.classes_))


35/35 ━━━━━━━━━━━━━━━━━━━━ 5s 139ms/step

Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00       159
     disgust       0.30      0.47      0.36       159
        fear       0.00      0.00      0.00       159
   happiness       0.19      0.89      0.31       160
     neutral       0.00      0.00      0.00       160
     sadness       0.20      0.15      0.17       159
    surprise       0.00      0.00      0.00       160

    accuracy                           0.22      1116
   macro avg       0.10      0.22      0.12      1116
weighted avg       0.10      0.22      0.12      1116



/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/celinewu/anaconda3/envs/2B/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()